In [1]:
include("../../L-BFGS/OracleFunction.jl")
include("../../L-BFGS/LBFGS.jl")
include("../../utilities/genFunc.jl")
using .LBFGS
using .OracleFunction
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = joinpath("../", "results/LBFGS/ill_conditioned/")
mkpath(baseDir);

In [2]:
# parameters for exact LS vs AWLS vs quadratic function 
λ        = 10^-12
ϵ        = 10^-14
memsize  = 7
maxIters = 1:200
m        = 1000
n        = 20
num_trials = 20

gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
non_quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]),
        (x) -> inv(norm(gf[:X_hat] * x - gf[:y_hat])) * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat])^2,
        (x) -> 2 * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
ls = LeastSquaresF(gf)

LeastSquaresF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}(OracleF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}([0.2592586022640595 0.6963535943217778 … 0.2686596970120587 -0.36678046257922436; 0.514702105566236 -0.7473274556601393 … 0.19887814981786733 0.8663661527822621; … ; 0.0 0.0 … 1.0e-12 0.0; 0.0 0.0 … 0.0 1.0e-12], [7.689547521112595, -23.188865118086323, 13.727391619018281, -4.242513852908478, 23.90743053982707, 1.8343747758964255, -1.159884201591351, 2.216694337124031, 2.730975021441503, -19.180979473592664  …  -6.088266825619777e-13, 1.1943252953272254e-13, 4.048371886383304e-13, -6.926115820555791e-13, 3.8752674617747763e

In [5]:
### residual, relative error and norm of gradient with respect to iterations with exact line search
using Statistics, CSV

# Preallocate arrays
num_iterations = length(maxIters)

gradients = zeros(num_trials, num_iterations)
residuals = zeros(num_trials, num_iterations)
relative_errors = zeros(num_trials, num_iterations)

for trial in 1:num_trials

    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    for (i, maxIter) in enumerate(maxIters)
        t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

        relative_errors[trial, i] = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residuals[trial, i] = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])
        gradients[trial, i] = norm(t[:grad])
    end

    println("Done trial ", trial)
    
end

# Calculate mean and standard deviation
mean_relative = mean(relative_errors, dims=1)'
std_relative = std(relative_errors, dims=1)'
mean_residual = mean(residuals, dims=1)'
std_residual = std(residuals, dims=1)'
mean_gradient = mean(gradients, dims=1)'
std_gradient = std(gradients, dims=1)'


# Write results to CSV
outputvsc = joinpath(baseDir, "statisticsLBFGS-iterations-m" * string(m) * "n" * string(n) * "--error-norm.csv");

accData = Dict(
    :lambda        => Array{Float64}(undef, 0),
    :epsilon       => Array{Float64}(undef, 0),
    :memsize       => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :mean_relative => Array{Float64}(undef, 0),
    :std_relative  => Array{Float64}(undef, 0),
    :mean_residual => Array{Float64}(undef, 0),
    :std_residual  => Array{Float64}(undef, 0),
    :mean_gradient => Array{Float64}(undef, 0),
    :std_gradient  => Array{Float64}(undef, 0)
)

# create dataframe with columns from arrays
for maxIter ∈ maxIters
    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:maxiterations], maxIter)
    push!(accData[:mean_relative], mean_relative[maxIter])
    push!(accData[:std_relative], std_relative[maxIter])
    push!(accData[:mean_residual], mean_residual[maxIter])
    push!(accData[:std_residual], std_residual[maxIter])
    push!(accData[:mean_gradient], mean_gradient[maxIter])
    push!(accData[:std_gradient], std_gradient[maxIter])

end


CSV.write(outputvsc, accData)


Done trial 1
Done trial 2
Done trial 3
Done trial 4
Done trial 5
Done trial 6
Done trial 7
Done trial 8
Done trial 9
Done trial 10
Done trial 11
Done trial 12
Done trial 13
Done trial 14
Done trial 15
Done trial 16
Done trial 17
Done trial 18
Done trial 19
Done trial 20


"../results/LBFGS/ill_conditioned/statisticsLBFGS-iterations-m1000n20--error-norm.csv"

In [3]:
### relative error with respect to iterations and memeory size
# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-iterations-m" * string(m) * "n" * string(n) * "--memsize.csv");
accData = Dict(


    :memsize       => Array{Int64}(undef, 0),
    :iterations    => Array{Int64}(undef, 0),
    :maxiterations => Array{Int64}(undef, 0),
    :relative      => Array{Float64}(undef, 0),
    :residual      => Array{Float64}(undef, 0),
    :gradient      => Array{Float64}(undef, 0)
    )

memsizes = [1 3 5 7 9 11]

for memsize ∈ memsizes
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)
    for maxIter ∈ maxIters

        t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

        relative_error = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residual = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])

        iterations = maxIter - t[:RemainingEvaluations]

        push!(accData[:memsize], memsize)
        push!(accData[:iterations], iterations)
        push!(accData[:maxiterations], maxIter)
        push!(accData[:relative], relative_error)
        push!(accData[:residual], residual)
        push!(accData[:gradient], norm(t[:grad]))
        flush(stdout)
    end
end

CSV.write(outputvsc, DataFrame(accData));